In [77]:
import pandas as pd
import stringEncoder
import seaborn as sn
import numpy as np

from sklearn.preprocessing import Imputer
from sklearn.utils import shuffle
from sklearn import tree
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

Import training data

In [78]:
#an training mtrix contain all newly registered airbnb users.
df = pd.read_csv('../data/train_users_2.csv')

import testing(predict) data

In [79]:
test_df = pd.read_csv('../data/test_users.csv')
test_id = test_df.id

test_df.drop(["id","date_first_booking"], 1, inplace=True)
df.drop(["id","date_first_booking"], 1, inplace=True)
df = shuffle(df)

extract lable types

In [80]:
y_train = df.country_destination

result_lable =[]
for data in y_train:
    if data not in result_lable:
        result_lable.append(data)
print result_lable

['NDF', 'other', 'US', 'AU', 'DE', 'FR', 'IT', 'CA', 'GB', 'ES', 'NL', 'PT']


Replace unknow with NaN, then fill na with most frequent data

In [85]:
df.replace('-unknown-', np.nan, inplace=True)
df = df.apply(lambda x:x.fillna(x.value_counts().index[0]))
df.shape

(213451, 14)

In [82]:
test_df.replace('-unknown-', np.nan, inplace=True)
test_df = test_df.apply(lambda x:x.fillna(x.value_counts().index[0]))
test_df.shape

(62096, 13)

one hot encoding non numerical features on training set

In [66]:
# one-hot-encode all the categorical data:
# gender signup_method language affiliate_channel 
# affiliate_provider first_affiliate_tracked signup_app	
# first_device_type first_browser	country_destination
oh_gender = pd.get_dummies(df.gender, prefix="gender")
oh_language = pd.get_dummies(df.language, prefix="language")
oh_signup_method = pd.get_dummies(df.signup_method, prefix="signup_method")
oh_affiliate_channel = pd.get_dummies(df.affiliate_channel, prefix="affiliate_channel")
oh_affiliate_provider = pd.get_dummies(df.affiliate_provider, prefix="affiliate_provider")
oh_first_affiliate_tracked = pd.get_dummies(df.first_affiliate_tracked, prefix="first_affiliate_tracked")
oh_signup_app = pd.get_dummies(df.signup_app, prefix="signup_app")
oh_first_device_type = pd.get_dummies(df.first_device_type, prefix="first_device_type")
oh_first_browser = pd.get_dummies(df.first_browser, prefix="first_browser")

#dropped date_account_created date_first_booking add back later
df.drop(['date_account_created','gender', 'signup_method',
         'language', 'affiliate_channel', 
         'affiliate_provider', 'first_affiliate_tracked', 
         'signup_app', 'first_device_type', 'first_browser','country_destination'],1,inplace=True)
# from the traning set
x_train = pd.concat([df,oh_gender, oh_language, 
                oh_signup_method, oh_affiliate_channel, 
                oh_affiliate_provider,
                oh_first_affiliate_tracked, oh_signup_app,
                oh_first_device_type, oh_first_browser],axis=1)

In [71]:
x_train.head(5)


,timestamp_first_active,age,signup_flow,gender_FEMALE,gender_MALE,gender_OTHER,language_ca,language_cs,language_da,language_de,...,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
47238,2.010000e+13,40.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163299,2.010000e+13,27.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47431,2.010000e+13,29.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64181,2.010000e+13,24.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176839,2.010000e+13,30.0,25,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


one hot encoding on testing data

In [68]:
# one-hot-encode all the categorical data:
# gender signup_method language affiliate_channel 
# affiliate_provider first_affiliate_tracked signup_app	
# first_device_type first_browser	country_destination
oh_gender = pd.get_dummies(test_df.gender, prefix="gender")
oh_language = pd.get_dummies(test_df.language, prefix="language")
oh_signup_method = pd.get_dummies(test_df.signup_method, prefix="signup_method")
oh_affiliate_channel = pd.get_dummies(test_df.affiliate_channel, prefix="affiliate_channel")
oh_affiliate_provider = pd.get_dummies(test_df.affiliate_provider, prefix="affiliate_provider")
oh_first_affiliate_tracked = pd.get_dummies(test_df.first_affiliate_tracked, prefix="first_affiliate_tracked")
oh_signup_app = pd.get_dummies(test_df.signup_app, prefix="signup_app")
oh_first_device_type = pd.get_dummies(test_df.first_device_type, prefix="first_device_type")
oh_first_browser = pd.get_dummies(test_df.first_browser, prefix="first_browser")

#dropped date_account_created date_first_booking add back later
test_df.drop(['date_account_created','gender', 'signup_method',
         'language', 'affiliate_channel', 
         'affiliate_provider', 'first_affiliate_tracked', 
         'signup_app', 'first_device_type', 'first_browser'],1,inplace=True)
# from the traning set
x_test = pd.concat([test_df,oh_gender, oh_language, 
                oh_signup_method, oh_affiliate_channel, 
                oh_affiliate_provider,
                oh_first_affiliate_tracked, oh_signup_app,
                oh_first_device_type, oh_first_browser],axis=1)

In [75]:
x_test.shape

(62096, 107)

## Radial Basis Function (RBF) kernel SVM



In [ ]:
from sklearn.svm import SVC
svc = SVC(gamma=2, C=1)
svc.fit(x_train,y_train)